In [4]:
import cv2
from numpy import *
from scipy import ndimage
from copy import deepcopy

In [5]:
#Obtaining the images for the application

scaling = 0.2
img1 = cv2.imread('1_2.jpg')
img2 = cv2.imread('1_3.jpg')
img3 = cv2.imread('1_4.jpg')
img4 = cv2.imread('1_5.jpg')


img1 =  cv2.resize(img1,(0,0), fx = scaling, fy = scaling)
img2 =  cv2.resize(img2,(0,0), fx = scaling, fy = scaling)
img3 =  cv2.resize(img3,(0,0), fx = scaling, fy = scaling)
img4 =  cv2.resize(img4,(0,0), fx = scaling, fy = scaling)




gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
gray3 = cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
gray4 = cv2.cvtColor(img4,cv2.COLOR_BGR2GRAY)




In [6]:
#Obtaining the distance transform used in warping the two images.

def obtainDistanceTransform(img): 
    rows = img.shape[0]
    cols = img.shape[1]
    ret = zeros([rows,cols,3])
    a =  ones([rows,cols])
    b = zeros([rows+2,cols+2])
    b[1:-1,1:-1] = a
    
    c = ndimage.distance_transform_edt(b)
    d = c[1:-1,1:-1]
    [ret[:,:,0],ret[:,:,1],ret[:,:,2]] = [d,d,d] 

    return ret

In [7]:
#A function to interpolate the distance transform of the image.
def interpolate_dist(img):
    rows,cols = img.shape[0],img.shape[1]
    dst = img.copy()
    for i in range(1,rows-1): 
        for j in range(1,cols-1): 
            if(img[i,j].tolist()==[1,1,1]):
                isolate = img[i-1:i+2,j-1:j+2].reshape(9,3)
                for m in range(9): 
                    if(isolate[m].tolist()!=[1,1,1]):
                        dst[i,j] = isolate[m]
   
    return dst

In [8]:
#A function to obtain keypoints from img2 to img1
def MatchKeypoints(img1,img2): 
    gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    bf = cv2.BFMatcher()
    kp1,des1 = sift.detectAndCompute(gray1,None)
    kp2,des2 = sift.detectAndCompute(gray2,None)
    matches = bf.knnMatch(des2,des1,k=2)
    
    good = findgoodmatches(matches)
    
    
    src_pts = float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = float32([kp1[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    
    return src_pts,dst_pts
    

In [9]:
#A function to Convert src_pts and dst_pts to a usable format. 
def ConvertArray(src_pts,dst_pts): 
    src = []
    des = []
    for i in src_pts: 
        k = i.ravel().tolist()
        k.append(1)
        src.append(k)

    for i in dst_pts: 
        k = i.ravel().tolist()
        k.append(1)
        des.append(k)

    src = asarray(src)
    des = asarray(des)
    return src,des





In [10]:
#A function to warp an image with a homography matrix
def warp_image(img2,homo):
    all_src = []
 
    for i in range(img2.shape[0]):
        for j in range(img2.shape[1]): 
            all_src.append([j,i,1])
    all_src = asarray(all_src)
    all_src_trans = all_src.T
    all_des_trans = normalize(dot(homo,all_src_trans))
    all_des = all_des_trans.T
    all_des = all_des.astype('int32')
    return all_src,all_des
    

In [11]:
#A function to interpolate the obtained panaroma.
def interpolate(img):
    rows,cols = img.shape[0],img.shape[1]
    dst = img.copy()
    for i in range(1,rows-1): 
        for j in range(1,cols-1): 
            if(img[i,j].tolist()==[0,0,0]):
                isolate = img[i-1:i+2,j-1:j+2].reshape(9,3)
                for m in range(9): 
                    if(isolate[m].tolist()!=[0,0,0]):
                        dst[i,j] = isolate[m]
   
    return dst.astype('uint8')
                
    

In [12]:
#A function to normalize the output obtained in each round of RANSAC.
def normalize(mat): 
    if(0 not in mat[-1]):
        return mat/mat[-1]
    else: 
        return zeros([3,len(mat[1])])

In [13]:
#A function to estimate the homography for a pair of images with given threshold. 
def estimatehomography(img1,img2,threshold):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    src_pts, dst_pts = MatchKeypoints(img1,img2)
    
    src, dst = ConvertArray(src_pts,dst_pts)
    
    homo = ransac(src,dst,threshold)
    
    return homo
    ##Now apply RANSAC on the image. 
    
    
    

In [14]:
#A function to calculate the value of error in RANSAC. 
def calculate_norm(array1,array2): 
    diff = subtract(array1,array2)
    return (diff[0]**2+diff[1]**2+diff[2]**2)**0.5

In [15]:
#A function to calculate homography between 4 points. 
def calculatehomography(rand_pts,src,dst): 
   
    A = zeros([8,9])
    for i in range(4):
            pt = int(rand_pts[i])
            s = src[pt]
            d = dst[pt]
            A[2*i]   = [-s[0],-s[1],-s[2],0,0,0,d[0]*s[0],d[0]*s[1],d[0]*s[2]]
            A[2*i+1] = [0,0,0,-s[0],-s[1],-s[2],d[1]*s[0],d[1]*s[1],d[1]*s[2]]
            
    u, s, v = linalg.svd(A)

    #reshape the min singular value into a 3 by 3 matrix
    h = reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h
    

In [16]:
#The ransac function, which returns the homography matrix
def ransac(src,dst,threshold):
    #First we choose any 4 random points and estimate the homography
    
    max_cnt = 0
    max_inlier = []
    number = len(src)
    threshold_inliers = int(0.90*len(src))
    best_homo = zeros([3,3])
    for ite in range(7000):
       
        rand_pts = random.randint(len(src),size = (4,)) 
        inliers = []
        final = []
        H = calculatehomography(rand_pts,src,dst)
        transposed_src = src.T
        transposed_des = normalize(dot(H,transposed_src))
        inliers = []
        expected_des = dst.T
        distance = calculate_norm(expected_des,transposed_des)
        for i in range(len(distance)): 
            if(distance[i]<threshold):
                inliers.append(i)
            if(len(inliers)>threshold_inliers): 
                max_inlier = inliers
                max_cnt = len(inliers)
                best_homo = H
                break
            elif(len(inliers)>max_cnt):
                max_cnt = len(inliers)
                max_inlier = inliers 
                best_homo = H
                
    return best_homo
        
                
    




    
        
                
                
        
        
    
        
    
        
        
    

In [17]:
#A function to find good matched between given set of keypoints
def findgoodmatches(matches):
    good = []
    for m,n in matches: 
        if m.distance<0.75*n.distance:
            good.append(m)
    return good


In [18]:
#The code for stitching 4 images together using homography matrix
def StitchHomography(img1,img2,img3,img4):
    src_pts,des_pts = MatchKeypoints(img1,img2)
    H21 = estimatehomography(img1,img2,3)

    src_pts,des_pts = MatchKeypoints(img2,img3)
    H32 = estimatehomography(img2,img3,3)

    src_pts,des_pts = MatchKeypoints(img3,img4)
    H43 = estimatehomography(img3,img4,3)

    #H21 = estimatehomography(img1,img2,3)
    #H32 = estimatehomography(img2,img3,3)
    H31 = dot(H32,H21)
    H41 = dot(H43,H31)

    dist1 = obtainDistanceTransform(img1)
    dist2 = obtainDistanceTransform(img2) 
    dist3 = obtainDistanceTransform(img3)
    dist4 = obtainDistanceTransform(img4)

    src2,des2 = warp_image(img2,H21)
    src3,des3 = warp_image(img3,H31)
    src4,des4 = warp_image(img4,H41)

    mini_col = 0
    mini_row = 0
    max_row = 0
    max_col = 0
    for i in range(len(des3)): 
        col = des3[i][0]
        row = des3[i][1]

        if(row<mini_row):
            mini_row = row

        if(col<mini_col):
            mini_col = col

        if(row>max_row):
            max_row = row

        if(col>max_col):
            max_col = col

    for i in range(len(des2)):
        col = des2[i][0]
        row = des2[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row


    for i in range(len(des4)):
        col = des4[i][0]
        row = des4[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row

    final_rows = max_row-mini_row+1
    final_cols = max_col-mini_col+1

    final_image = zeros([final_rows,final_cols,3])
    I1 = zeros([final_rows,final_cols,3])
    I2 = zeros([final_rows,final_cols,3])
    I3 = zeros([final_rows,final_cols,3])
    I4 = zeros([final_rows,final_cols,3])

    distance_1 = ones([final_rows,final_cols,3])
    distance_2 = ones([final_rows,final_cols,3])
    distance_3 = ones([final_rows,final_cols,3])
    distance_4 = ones([final_rows,final_cols,3])



    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            I1[i-mini_row,j-mini_col] = img1[i,j]
            distance_1[i-mini_row,j-mini_col] = dist1[i,j]



    for i in range(len(des4)):
        row = des4[i][1]
        col = des4[i][0]
        val1 = img4[int(src4[i][1]),int(src4[i][0])]
        val2 = dist4[int(src4[i][1]),int(src4[i][0])]
        distance_4[row-mini_row,col-mini_col] = val2
        I4[row-mini_row,col-mini_col] = val1


    for i in range(len(des3)):
        row = des3[i][1]
        col = des3[i][0]
        val1 = img3[int(src3[i][1]),int(src3[i][0])]
        val2 = dist3[int(src3[i][1]),int(src3[i][0])]
        I3[row-mini_row,col-mini_col] = val1
        distance_3[row-mini_row,col-mini_col] = val2

    for i in range(len(des2)):
        row = des2[i][1]
        col = des2[i][0]
        val1 = img2[int(src2[i][1]),int(src2[i][0])]
        val2 = dist2[int(src2[i][1]),int(src2[i][0])]
        I2[row-mini_row,col-mini_col] = val1
        distance_2[row-mini_row,col-mini_col] = val2



    I1 = interpolate(I1.astype('uint8'))
    I2 = interpolate(I2.astype('uint8'))
    I3 = interpolate(I3.astype('uint8'))
    I4 = interpolate(I4.astype('uint8'))
    
    distance_1 = interpolate_dist(distance_1)
    distance_2 = interpolate_dist(distance_2)
    distance_3 = interpolate_dist(distance_3)
    distance_4 = interpolate_dist(distance_4)
    
    
    final_image = (I1*distance_1+I2*distance_2+I3*distance_3+I4*distance_4)/(distance_1+distance_2+distance_3+distance_4)
    final_image = final_image.astype('uint8')
    return final_image

    





In [19]:
#Stitching using the inbuilt function. 
def StitchInbuilt(img1,img2,img3,img4):
    
    src_pts,des_pts = MatchKeypoints(img1,img2)
    H21,mask = cv2.findHomography(src_pts,des_pts,cv2.RANSAC,3.0)

    src_pts,des_pts = MatchKeypoints(img2,img3)
    H32,mask = cv2.findHomography(src_pts,des_pts,cv2.RANSAC,3.0)

    src_pts,des_pts = MatchKeypoints(img3,img4)
    H43,mask = cv2.findHomography(src_pts,des_pts,cv2.RANSAC,3.0)

    #H21 = estimatehomography(img1,img2,3)
    #H32 = estimatehomography(img2,img3,3)
    H31 = dot(H32,H21)
    H41 = dot(H43,H31)

    dist1 = obtainDistanceTransform(img1)
    dist2 = obtainDistanceTransform(img2) 
    dist3 = obtainDistanceTransform(img3)
    dist4 = obtainDistanceTransform(img4)

    src2,des2 = warp_image(img2,H21)
    src3,des3 = warp_image(img3,H31)
    src4,des4 = warp_image(img4,H41)

    mini_col = 0
    mini_row = 0
    max_row = 0
    max_col = 0
    for i in range(len(des3)): 
        col = des3[i][0]
        row = des3[i][1]

        if(row<mini_row):
            mini_row = row

        if(col<mini_col):
            mini_col = col

        if(row>max_row):
            max_row = row

        if(col>max_col):
            max_col = col

    for i in range(len(des2)):
        col = des2[i][0]
        row = des2[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row


    for i in range(len(des4)):
        col = des4[i][0]
        row = des4[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row

    final_rows = max_row-mini_row+1
    final_cols = max_col-mini_col+1

    final_image = zeros([final_rows,final_cols,3])
    I1 = zeros([final_rows,final_cols,3])
    I2 = zeros([final_rows,final_cols,3])
    I3 = zeros([final_rows,final_cols,3])
    I4 = zeros([final_rows,final_cols,3])

    distance_1 = ones([final_rows,final_cols,3])
    distance_2 = ones([final_rows,final_cols,3])
    distance_3 = ones([final_rows,final_cols,3])
    distance_4 = ones([final_rows,final_cols,3])



    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            I1[i-mini_row,j-mini_col] = img1[i,j]
            distance_1[i-mini_row,j-mini_col] = dist1[i,j]



    for i in range(len(des4)):
        row = des4[i][1]
        col = des4[i][0]
        val1 = img4[int(src4[i][1]),int(src4[i][0])]
        val2 = dist4[int(src4[i][1]),int(src4[i][0])]
        distance_4[row-mini_row,col-mini_col] = val2
        I4[row-mini_row,col-mini_col] = val1


    for i in range(len(des3)):
        row = des3[i][1]
        col = des3[i][0]
        val1 = img3[int(src3[i][1]),int(src3[i][0])]
        val2 = dist3[int(src3[i][1]),int(src3[i][0])]
        I3[row-mini_row,col-mini_col] = val1
        distance_3[row-mini_row,col-mini_col] = val2

    for i in range(len(des2)):
        row = des2[i][1]
        col = des2[i][0]
        val1 = img2[int(src2[i][1]),int(src2[i][0])]
        val2 = dist2[int(src2[i][1]),int(src2[i][0])]
        I2[row-mini_row,col-mini_col] = val1
        distance_2[row-mini_row,col-mini_col] = val2



    I1 = interpolate(I1.astype('uint8'))
    I2 = interpolate(I2.astype('uint8'))
    I3 = interpolate(I3.astype('uint8'))
    I4 = interpolate(I4.astype('uint8'))
    
    distance_1 = interpolate_dist(distance_1)
    distance_2 = interpolate_dist(distance_2)
    distance_3 = interpolate_dist(distance_3)
    distance_4 = interpolate_dist(distance_4)
    
    
    final_image = (I1*distance_1+I2*distance_2+I3*distance_3+I4*distance_4)/(distance_1+distance_2+distance_3+distance_4)
    final_image = final_image.astype('uint8')
    return final_image

    

In [20]:
#Q1 part(b)
#A code to calculate the homography matrix after applying RANSAC 

threshold = 3
H1 = estimatehomography(img1,img2,3)
H2 = estimatehomography(img2,img3,3)
H3 = estimatehomography(img4,img3,3)

print "homography between 2 and 1"
print H1

print "homography between 3 and 2"
print H2

print "homography between 4 and 3"
print H3

homography between 2 and 1
[[ 7.82260487e-01 -1.97881080e-02  1.02799204e+02]
 [-3.70048889e-02  9.33374333e-01  1.25577069e+01]
 [-5.32969675e-04  2.77674673e-05  1.00000000e+00]]
homography between 3 and 2
[[ 8.26846145e-01 -3.72655137e-02  8.90241623e+01]
 [-2.82350914e-02  9.22339828e-01  1.74276493e+01]
 [-4.25810310e-04 -4.68542879e-05  1.00000000e+00]]
homography between 4 and 3
[[ 1.32935553e+00 -1.47731596e-02 -1.60208228e+02]
 [ 1.07105532e-01  1.21609079e+00 -4.09424223e+01]
 [ 8.71564275e-04 -2.17236977e-05  1.00000000e+00]]


In [21]:
#Q1 part(c) 
#Code to stitch 4 input images together.

final_image = StitchHomography(img1,img2,img3,img4)
cv2.imshow('final_image',final_image)
cv2.waitKey(0)


-1

In [82]:
#Q1 part(d)
#A code to obtain results using inbuilt homography

final_image = StitchInbuilt(img1,img2,img3,img4)
cv2.imshow('final_image',final_image)
cv2.waitKey(0)


-1